# Ch22: 除錯技術 - 詳解範例

本 Notebook 包含 **4 個完整的實務案例**，展示除錯技術在實際應用中的最佳實踐。

## 📋 案例清單
1. 除錯遞迴函式（階乘計算錯誤）
2. 除錯迴圈邏輯（無窮迴圈與邊界錯誤）
3. 除錯資料結構（IndexError, KeyError）
4. 效能問題分析（使用 logging 追蹤執行時間）

---

## 案例 1: 除錯遞迴函式

### 情境
計算階乘的遞迴函式有 bug，導致無窮遞迴或結果錯誤。

### 要處理的錯誤
- 遞迴沒有正確終止
- 遞迴參數沒有正確遞減
- 邊界條件錯誤

### 學習重點
- 使用 `breakpoint()` 設定中斷點
- 使用 pdb 檢查變數值
- 追蹤遞迴呼叫鏈

In [ ]:
# 步驟 1: 有 bug 的階乘函式

def factorial_buggy(n):
    """
    計算階乘（有 bug）
    Bug: 遞迴沒有遞減，會無窮遞迴
    """
    print(f"factorial_buggy({n}) 被呼叫")
    
    if n == 0:
        return 1
    
    # Bug: 沒有遞減 n，會無窮遞迴
    return n * factorial_buggy(n)  # 應該是 factorial_buggy(n-1)

# 測試（會導致 RecursionError）
try:
    result = factorial_buggy(5)
    print(f"結果: {result}")
except RecursionError as e:
    print(f"\n❌ 錯誤: RecursionError - 超過最大遞迴深度")
    print("原因: 遞迴沒有正確終止條件")

In [ ]:
# 步驟 2: 使用 print debugging 追蹤

def factorial_debug_v1(n, depth=0):
    """
    使用 print 追蹤遞迴過程
    """
    indent = "  " * depth
    print(f"{indent}→ factorial({n}) 開始")
    
    if n == 0:
        print(f"{indent}← 到達終止條件，返回 1")
        return 1
    
    # 修正: n-1
    result = n * factorial_debug_v1(n - 1, depth + 1)
    print(f"{indent}← factorial({n}) 返回 {result}")
    return result

# 測試
print("=== Print Debugging 輸出 ===")
result = factorial_debug_v1(5)
print(f"\n最終結果: {result}")  # 應該是 120

In [ ]:
# 步驟 3: 使用 logging 模組

import logging

# 設定 logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(levelname)s - %(message)s',
    force=True
)

logger = logging.getLogger(__name__)

def factorial_logging(n, depth=0):
    """
    使用 logging 追蹤遞迴過程
    """
    indent = "  " * depth
    logger.debug(f"{indent}factorial({n}) - 深度 {depth}")
    
    if n < 0:
        logger.error(f"{indent}無效輸入: n={n} (必須 >= 0)")
        raise ValueError("n 必須 >= 0")
    
    if n == 0:
        logger.debug(f"{indent}終止條件達成")
        return 1
    
    result = n * factorial_logging(n - 1, depth + 1)
    logger.info(f"{indent}factorial({n}) = {result}")
    return result

# 測試
print("\n=== Logging 輸出 ===")
result = factorial_logging(4)
print(f"\n最終結果: {result}")

In [ ]:
# 步驟 4: 使用 breakpoint() 除錯（需在命令列執行）

def factorial_pdb(n):
    """
    使用 pdb 除錯的階乘函式
    
    除錯提示:
    1. 取消註解 breakpoint()
    2. 在命令列執行此程式
    3. 在 pdb 中使用:
       - p n         # 查看 n 的值
       - n           # 執行下一行
       - s           # 進入遞迴呼叫
       - w           # 查看呼叫堆疊
       - c           # 繼續執行
    """
    if n == 0:
        return 1
    
    # breakpoint()  # 取消註解後在這裡暫停
    result = n * factorial_pdb(n - 1)
    return result

# 測試
result = factorial_pdb(5)
print(f"✅ 修正後的結果: {result}")  # 120

print("\n💡 提示: 在命令列執行此腳本並取消註解 breakpoint() 以體驗 pdb 除錯")

### 案例 1 總結

**除錯遞迴函式的關鍵步驟**:

1. **識別問題**: 無窮遞迴 → 檢查終止條件與遞減
2. **Print Debugging**: 追蹤每次呼叫的參數與返回值
3. **Logging**: 使用不同級別記錄執行流程
4. **pdb**: 在關鍵點暫停，檢查堆疊

**常見遞迴錯誤**:
- ✅ 忘記遞減參數
- ✅ 終止條件錯誤
- ✅ 沒有處理邊界情況（負數）

---

## 案例 2: 除錯迴圈邏輯

### 情境
找出質數的函式有多個 bug:
- 邊界條件錯誤（1 不是質數）
- 迴圈範圍錯誤（沒有檢查到正確的除數）
- 邏輯錯誤（提前終止）

### 學習重點
- 追蹤迴圈變數
- 檢查邊界條件
- 使用 logging 記錄迴圈狀態

In [ ]:
# 步驟 1: 有多個 bug 的質數檢查函式

def is_prime_buggy(n):
    """
    檢查是否為質數（有多個 bug）
    
    Bug 1: 沒有處理 n < 2 的情況
    Bug 2: range 範圍錯誤（應該到 sqrt(n)）
    Bug 3: 邏輯錯誤（應該找不能整除的）
    """
    for i in range(2, n):  # Bug 2: 範圍太大，效率低
        if n % i != 0:  # Bug 3: 邏輯錯誤
            return True
    return False

# 測試
print("=== 測試有 bug 的版本 ===")
test_numbers = [1, 2, 3, 4, 5, 9, 11, 15]
for num in test_numbers:
    result = is_prime_buggy(num)
    print(f"is_prime({num}) = {result}")

print("\n❌ 觀察到的問題:")
print("1. is_prime(1) = False（正確，但沒有特別處理）")
print("2. is_prime(4) = True（錯誤！4 不是質數）")
print("3. is_prime(9) = True（錯誤！9 不是質數）")

In [ ]:
# 步驟 2: 使用科學除錯法

# 重現問題：建立最小測試案例
print("=== 步驟 1: 重現問題 ===")
print(f"is_prime_buggy(4) = {is_prime_buggy(4)}")
print("期望: False，實際: True")
print("\n問題：4 不是質數（4 = 2 × 2），但函式返回 True")

In [ ]:
# 步驟 3: 隔離問題

import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(message)s',
    force=True
)

def is_prime_debug(n):
    """
    使用 logging 追蹤迴圈邏輯
    """
    logging.debug(f"\n檢查 {n} 是否為質數")
    
    for i in range(2, n):
        remainder = n % i
        logging.debug(f"  i={i}: {n} % {i} = {remainder}")
        
        if remainder != 0:
            logging.debug(f"    → {n} 不能被 {i} 整除，返回 True")
            return True
    
    logging.debug("  迴圈結束，返回 False")
    return False

print("=== 步驟 2: 隔離問題（使用 logging 追蹤）===")
result = is_prime_debug(4)
print(f"\n結果: {result}")
print("\n💡 發現問題: 當 i=2 時，4 % 2 = 0（能整除），但邏輯寫成 != 0 時返回 True")
print("   應該是: 如果能整除（== 0），就不是質數，返回 False")

In [ ]:
# 步驟 4: 假設與驗證

print("=== 步驟 3: 提出假設 ===")
print("假設 1: 邏輯應該是 'if n % i == 0: return False'")
print("假設 2: 需要處理 n < 2 的特殊情況")
print("假設 3: 可以優化範圍到 sqrt(n)")

import math

def is_prime_fixed(n):
    """
    修正後的質數檢查（含優化）
    """
    # 修正 1: 處理邊界條件
    if n < 2:
        return False
    
    # 修正 2: 優化範圍到 sqrt(n) + 1
    for i in range(2, int(math.sqrt(n)) + 1):
        # 修正 3: 邏輯修正（能整除就不是質數）
        if n % i == 0:
            return False
    
    return True

print("\n=== 步驟 4: 驗證修正 ===")
test_cases = [
    (1, False),
    (2, True),
    (3, True),
    (4, False),
    (5, True),
    (9, False),
    (11, True),
    (15, False),
    (17, True)
]

all_passed = True
for num, expected in test_cases:
    result = is_prime_fixed(num)
    status = "✓" if result == expected else "✗"
    if result != expected:
        all_passed = False
    print(f"{status} is_prime({num}) = {result} (期望: {expected})")

if all_passed:
    print("\n✅ 所有測試通過！修正成功")
else:
    print("\n❌ 仍有錯誤，需要進一步除錯")

### 案例 2 總結

**除錯迴圈邏輯的步驟**:

1. **重現**: 找出具體的錯誤案例（如 is_prime(4) = True）
2. **隔離**: 使用 logging 追蹤迴圈變數
3. **假設**: 分析邏輯錯誤（!= 應該是 ==）
4. **驗證**: 執行完整測試案例

**修正的三個 bug**:
- ✅ Bug 1: 邊界條件（n < 2）
- ✅ Bug 2: 迴圈範圍（優化到 sqrt(n)）
- ✅ Bug 3: 邏輯錯誤（!= 改成 ==）

---

## 案例 3: 除錯資料結構（IndexError, KeyError）

### 情境
學生成績管理系統有多個資料存取錯誤:
- 列表索引超出範圍（IndexError）
- 字典鍵不存在（KeyError）
- 未初始化的變數

### 學習重點
- 使用 try-except 處理資料存取錯誤
- 使用 logging 記錄錯誤資訊
- 使用 pdb 檢查資料結構狀態

In [ ]:
# 步驟 1: 有 bug 的成績管理系統

class GradeManagerBuggy:
    """
    成績管理系統（有多個 bug）
    """
    def __init__(self):
        self.students = {}  # {學號: {"name": 姓名, "grades": [成績列表]}}
    
    def add_student(self, student_id, name):
        self.students[student_id] = {"name": name, "grades": []}
    
    def add_grade(self, student_id, grade):
        """Bug 1: 沒有檢查 student_id 是否存在"""
        self.students[student_id]["grades"].append(grade)
    
    def get_average(self, student_id):
        """Bug 2: 沒有檢查 grades 是否為空"""
        grades = self.students[student_id]["grades"]
        return sum(grades) / len(grades)  # 可能 ZeroDivisionError
    
    def get_top_student(self):
        """Bug 3: 沒有檢查是否有學生"""
        averages = [(sid, self.get_average(sid)) for sid in self.students]
        return max(averages, key=lambda x: x[1])[0]  # 可能 ValueError

# 測試各種錯誤
print("=== 測試有 bug 的版本 ===")
manager = GradeManagerBuggy()

# 正常操作
manager.add_student("S001", "Alice")
manager.add_grade("S001", 85)
print(f"Alice 的平均: {manager.get_average('S001')}")

# 錯誤 1: KeyError
print("\n測試 Bug 1: 新增成績給不存在的學生")
try:
    manager.add_grade("S999", 90)  # S999 不存在
except KeyError as e:
    print(f"❌ KeyError: {e}")

# 錯誤 2: ZeroDivisionError
print("\n測試 Bug 2: 計算沒有成績的學生平均")
manager.add_student("S002", "Bob")
try:
    avg = manager.get_average("S002")  # Bob 沒有成績
except ZeroDivisionError:
    print(f"❌ ZeroDivisionError: 除以零（沒有成績）")

# 錯誤 3: ValueError
print("\n測試 Bug 3: 找出最高分學生（但有學生沒成績）")
try:
    top = manager.get_top_student()
except ZeroDivisionError:
    print(f"❌ ZeroDivisionError: Bob 沒有成績，無法計算平均")

In [ ]:
# 步驟 2: 使用 logging 追蹤錯誤

import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(levelname)s - %(message)s',
    force=True
)

class GradeManagerDebug:
    """
    使用 logging 追蹤的成績管理系統
    """
    def __init__(self):
        self.students = {}
        logging.info("成績管理系統初始化完成")
    
    def add_student(self, student_id, name):
        self.students[student_id] = {"name": name, "grades": []}
        logging.info(f"新增學生: {student_id} - {name}")
    
    def add_grade(self, student_id, grade):
        logging.debug(f"嘗試為 {student_id} 新增成績 {grade}")
        
        if student_id not in self.students:
            logging.error(f"學生 {student_id} 不存在")
            logging.debug(f"現有學生: {list(self.students.keys())}")
            raise KeyError(f"學生 {student_id} 不存在")
        
        self.students[student_id]["grades"].append(grade)
        logging.info(f"成功為 {student_id} 新增成績 {grade}")
    
    def get_average(self, student_id):
        logging.debug(f"計算 {student_id} 的平均成績")
        
        if student_id not in self.students:
            logging.error(f"學生 {student_id} 不存在")
            raise KeyError(f"學生 {student_id} 不存在")
        
        grades = self.students[student_id]["grades"]
        logging.debug(f"成績列表: {grades}")
        
        if len(grades) == 0:
            logging.warning(f"{student_id} 沒有成績，返回 None")
            return None
        
        avg = sum(grades) / len(grades)
        logging.info(f"{student_id} 的平均成績: {avg:.2f}")
        return avg

# 測試
print("\n=== 使用 logging 追蹤 ===")
manager = GradeManagerDebug()
manager.add_student("S001", "Alice")
manager.add_grade("S001", 85)
manager.add_grade("S001", 90)
avg = manager.get_average("S001")

print("\n測試錯誤情況:")
try:
    manager.add_grade("S999", 100)
except KeyError:
    pass

manager.add_student("S002", "Bob")
avg = manager.get_average("S002")  # 沒有成績

In [ ]:
# 步驟 3: 完整修正版本

class GradeManagerFixed:
    """
    修正後的成績管理系統
    """
    def __init__(self):
        self.students = {}
    
    def add_student(self, student_id, name):
        """新增學生"""
        if student_id in self.students:
            raise ValueError(f"學生 {student_id} 已存在")
        self.students[student_id] = {"name": name, "grades": []}
    
    def add_grade(self, student_id, grade):
        """新增成績（含錯誤處理）"""
        if student_id not in self.students:
            raise KeyError(f"學生 {student_id} 不存在")
        
        if not isinstance(grade, (int, float)):
            raise TypeError("成績必須是數字")
        
        if not 0 <= grade <= 100:
            raise ValueError("成績必須在 0-100 之間")
        
        self.students[student_id]["grades"].append(grade)
    
    def get_average(self, student_id):
        """計算平均（含錯誤處理）"""
        if student_id not in self.students:
            raise KeyError(f"學生 {student_id} 不存在")
        
        grades = self.students[student_id]["grades"]
        
        if len(grades) == 0:
            return None  # 或拋出例外，依業務邏輯決定
        
        return sum(grades) / len(grades)
    
    def get_top_student(self):
        """找出平均最高的學生"""
        if not self.students:
            raise ValueError("沒有學生資料")
        
        # 只計算有成績的學生
        students_with_grades = [
            (sid, self.get_average(sid))
            for sid in self.students
            if self.get_average(sid) is not None
        ]
        
        if not students_with_grades:
            raise ValueError("所有學生都沒有成績")
        
        return max(students_with_grades, key=lambda x: x[1])[0]

# 測試修正版本
print("\n=== 測試修正版本 ===")
manager = GradeManagerFixed()

# 正常操作
manager.add_student("S001", "Alice")
manager.add_student("S002", "Bob")
manager.add_student("S003", "Charlie")

manager.add_grade("S001", 85)
manager.add_grade("S001", 90)
manager.add_grade("S002", 75)
manager.add_grade("S002", 80)
manager.add_grade("S003", 95)

print(f"Alice 平均: {manager.get_average('S001')}")
print(f"Bob 平均: {manager.get_average('S002')}")
print(f"Charlie 平均: {manager.get_average('S003')}")
print(f"\n最高分學生: {manager.get_top_student()}")

print("\n✅ 所有功能正常運作！")

### 案例 3 總結

**除錯資料結構錯誤的步驟**:

1. **識別錯誤**: KeyError, IndexError, ZeroDivisionError
2. **使用 logging**: 追蹤資料狀態
3. **防禦性編程**: 加入檢查與驗證
4. **拋出有意義的例外**: 提供清楚的錯誤訊息

**修正的關鍵**:
- ✅ 檢查鍵是否存在（KeyError）
- ✅ 檢查列表是否為空（ZeroDivisionError）
- ✅ 驗證輸入資料（TypeError, ValueError）

---

## 案例 4: 效能問題分析

### 情境
一個數據處理程式執行很慢，需要找出效能瓶頸。

### 學習重點
- 使用 logging 記錄執行時間
- 使用 time 模組測量效能
- 找出效能瓶頸並優化

In [ ]:
# 步驟 1: 效能不佳的資料處理程式

import time

def process_data_slow(data):
    """
    效能不佳的資料處理（有效能問題）
    
    問題：
    1. 使用 list.append 在迴圈中建立新列表（慢）
    2. 重複計算相同的值
    3. 使用 in 檢查列表（O(n)）
    """
    results = []
    
    for i in range(len(data)):
        # 問題 1: 重複計算
        value = data[i] * 2 + data[i] ** 2
        
        # 問題 2: 使用 in 檢查列表（O(n)）
        if value not in results:
            results.append(value)
    
    return results

# 測試效能
test_data = list(range(1000))

start = time.time()
result = process_data_slow(test_data)
end = time.time()

print(f"慢版本執行時間: {end - start:.4f} 秒")
print(f"結果數量: {len(result)}")

In [ ]:
# 步驟 2: 使用 logging 追蹤執行時間

import logging
import time

logging.basicConfig(
    level=logging.INFO,
    format='%(message)s',
    force=True
)

def process_data_debug(data):
    """
    使用 logging 追蹤效能的版本
    """
    start_total = time.time()
    logging.info(f"開始處理 {len(data)} 筆資料")
    
    results = []
    
    start_loop = time.time()
    for i in range(len(data)):
        # 記錄每 100 筆的進度
        if i % 100 == 0:
            logging.debug(f"處理進度: {i}/{len(data)}")
        
        value = data[i] * 2 + data[i] ** 2
        
        # 測量 in 檢查的時間
        start_check = time.time()
        if value not in results:
            results.append(value)
        end_check = time.time()
        
        # 如果檢查時間過長，記錄警告
        if end_check - start_check > 0.001:
            logging.warning(
                f"檢查耗時過長: {end_check - start_check:.4f} 秒 "
                f"(results 長度: {len(results)})"
            )
    
    end_loop = time.time()
    logging.info(f"迴圈執行時間: {end_loop - start_loop:.4f} 秒")
    
    end_total = time.time()
    logging.info(f"總執行時間: {end_total - start_total:.4f} 秒")
    logging.info(f"結果數量: {len(results)}")
    
    return results

# 測試（使用較小的資料集以免輸出過多）
logging.getLogger().setLevel(logging.INFO)  # 只顯示 INFO
test_data = list(range(500))
result = process_data_debug(test_data)

print("\n💡 觀察：隨著 results 列表變長，'in' 檢查越來越慢")

In [ ]:
# 步驟 3: 優化版本

import time
import logging

def process_data_optimized(data):
    """
    優化後的資料處理
    
    優化:
    1. 使用 set 而非 list（O(1) vs O(n)）
    2. 使用 list comprehension
    3. 避免重複計算
    """
    start = time.time()
    logging.info(f"開始處理 {len(data)} 筆資料（優化版）")
    
    # 優化 1: 使用 set 去重（O(1) 查找）
    results_set = set()
    
    for value in data:
        # 優化 2: 簡化計算
        result = value * 2 + value ** 2
        results_set.add(result)
    
    # 轉回 list（如果需要）
    results = list(results_set)
    
    end = time.time()
    logging.info(f"總執行時間: {end - start:.4f} 秒")
    logging.info(f"結果數量: {len(results)}")
    
    return results

# 效能比較
test_data = list(range(2000))

print("=== 效能比較 ===")

# 慢版本
start = time.time()
result1 = process_data_slow(test_data)
time1 = time.time() - start
print(f"慢版本: {time1:.4f} 秒")

# 優化版本
logging.getLogger().setLevel(logging.WARNING)  # 減少輸出
start = time.time()
result2 = process_data_optimized(test_data)
time2 = time.time() - start
print(f"優化版本: {time2:.4f} 秒")

print(f"\n⚡ 效能提升: {time1 / time2:.1f}x")
print(f"結果相同: {set(result1) == set(result2)}")

### 案例 4 總結

**效能除錯的步驟**:

1. **測量基準**: 記錄原始執行時間
2. **找出瓶頸**: 使用 logging 追蹤各部分時間
3. **分析問題**: 
   - `in` 檢查列表是 O(n)，改用 set 是 O(1)
   - 重複計算可以快取
4. **優化並驗證**: 確保結果正確且效能提升

**常見效能問題**:
- ✅ 使用 list 而非 set 做去重
- ✅ 在迴圈中重複計算相同的值
- ✅ 使用 `in` 檢查大型列表

---

## 🎉 四個案例完成！

### 總結

您已完成 4 個實務案例：

1. **除錯遞迴函式** ✅
   - 使用 print/logging 追蹤遞迴流程
   - 使用 pdb 檢查呼叫堆疊
   - 修正終止條件與遞減錯誤

2. **除錯迴圈邏輯** ✅
   - 科學除錯法四步驟（重現→隔離→假設→驗證）
   - 使用 logging 追蹤迴圈變數
   - 修正邊界條件與邏輯錯誤

3. **除錯資料結構** ✅
   - 處理 KeyError, IndexError, ZeroDivisionError
   - 使用 logging 記錄資料狀態
   - 防禦性編程（加入驗證）

4. **效能問題分析** ✅
   - 使用 logging 記錄執行時間
   - 找出效能瓶頸
   - 優化資料結構（list → set）

### 核心技能

- ✅ 使用 print/logging 追蹤程式執行
- ✅ 使用 breakpoint() 設定中斷點
- ✅ 應用科學除錯法（重現→隔離→假設→驗證）
- ✅ 選擇合適的除錯工具（print vs logging vs pdb）
- ✅ 效能分析與優化

### 下一步

- 完成 **03-practice.ipynb** (8 題課堂練習)
- 完成 **04-exercises.ipynb** (12 題課後習題)
- 挑戰 **quiz.ipynb** (自我測驗)